<a href="https://colab.research.google.com/github/khbae/trading/blob/master/10_Value_at_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Value at Risk (VaR) and Expected Shortfall

## Financial Risks

* Credit Risk, default risk
* Market risk: risk due to changes in stock prices, interest rates, FX and commodity prices
* Operational risk: includes legal and political risks

## Value at Risk
"Value at risk (VaR) is a measure of the risk of loss for investments. It estimates how much a set of investments might lose (with a given probability), given normal market conditions, in a set time period such as a day. VaR is typically used by firms and regulators in the financial industry to gauge the amount of assets needed to cover possible losses."
* https://en.wikipedia.org/wiki/Value_at_risk

![alt text](https://upload.wikimedia.org/wikipedia/commons/6/64/VaR_diagram.JPG =500x)





Value-at-Risk (Var) is the single most important concept for market risk measurement.
* First used by J.P. Morgan for internal purposes during the 1990s
* Probability-based measure of loss potential for a company, fund, portfolio or strategy.
* Formal definition: **VaR is a monetary estimation of the loss that we expect to be exceeded with a given level of probability over a specific period of time.**
* So you will read a statement like:
    * The VaR for Star Fund is 2 million dollar for 1 day with probability of 0.05.
* What does this mean?
    * There is a 5 percent chance that Star Fund will lose at least 2 million dollar over the next one-day period 
    * 95 percent chance of loss less than 2 million dollars in one day
* So the probability level can be thought of as confidence level:
    * if it is reduced from 0.05 to 0.01, VaR number will be higher.
    
The user must, however, choose:
* The appropriate probability level (often 0.01 or 0.05) 0.01 is obviously a more conservative approach, and often chosen if you believe there is significant non-linear, “fat tail” in the distribution.
* Time period (usually a day for traders, two weeks for bank regulators )Obviously, the longer the time period, the greater the VaR numbereven if your probability level is unchanged (more things can go wrong in a week than in a day).

### Normal distribution Assumption

Suppose that portfolio's returns follow the normal distribution with expectation $\mu$ and standard deviation $\sigma$.

$$ \tilde{r} \sim N(\mu,\sigma^{2})$$

Let $P_0$ be portfolio's initial value. The portfolio value next period, $\tilde{X}$ is 

$$ \tilde{X} = P_0 \times (1+\tilde{r})  \sim N( P_0 (1+\mu) , P_0 \sigma^2)$$

Note that the portfolio value, $\tilde{X}$, follows a normal distribution. Let's normalize it:

$$ \frac{\tilde{X} - P_0(1+\mu)}{P_0 \sigma } \sim N(0,1) $$

Let $z_c$ be the $c$th quantile of $N(0,1)$. Then the portfolio's value at $c$th quantile is

$$ P_0 (1 + \mu + \sigma z_c)  $$

c% VaR would be

$$ VaR_c = P_0 - P_0 (1+ \mu + \sigma z_c) = -P_0(\mu+\sigma z_c) $$

In [0]:
# define VaR function
from scipy import stats

def VaR(P0,mu,sigma,c):
  zc = stats.norm.ppf(c,loc=0,scale=1)
  VaRout = -P0*(mu+sigma*zc)
  return VaRout

VaR(1,0.02,0.03,0.05)

0.02934560880854418

* https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html

In [0]:
!pip install -q pandas_datareader # pandas_datareader 패키지를 인스톨 하는 코드이다.

import pandas as pd
import pandas_datareader.data as web
import datetime # 원하는 날짜를 형성하여 만들 수 가 있다.
import numpy as np

start = datetime.datetime(2010, 1, 1)
end = datetime.datetime(2014, 1, 1)

aapl = web.DataReader("AAPL", "google", start, end) # APPLE 의 주가코드, 주식 데이터를 받아올 사이트, 그리고 시작날짜와 종료 날짜를 인자로 한다.

/usr/local/lib/python2.7/dist-packages/pandas_datareader/google/daily.py:40: UnstableAPIWarning: 
The Google Finance API has not been stable since late 2017. Requests seem
to fail at random. Failure is especially common when bulk downloading.

  warnings.warn(UNSTABLE_WARNING, UnstableAPIWarning)


In [0]:
mu = aapl['Close'].pct_change().mean()
sigma = aapl['Close'].pct_change().std()

P0 = 1000000

print(VaR(P0,mu,sigma,0.05))
print(VaR(P0,mu,sigma,0.01))


27690.25109372246
39624.36470615365


In [0]:
allvar = []
for i in range(1000):
  mont1 = pd.Series(np.random.normal(loc=mu,scale=sigma,size=1000))
  var = mont1.quantile(0.01) * P0
  allvar.append(var)
  #print(var)

np.mean(allvar)

-39270.54170565381

In [0]:
## Monte Carlo Simulation


27671.051522509497

In [0]:
# Change Distributional assumptions


27690.25109372246
28234.013433741762


## Expected Shortfall

"Expected shortfall (ES) is a risk measure—a concept used in the field of financial risk measurement to evaluate the market risk or credit risk of a portfolio."
* https://en.wikipedia.org/wiki/Expected_shortfall

Expected shortfall is also called 
* conditional value at risk (CVaR)
* average value at risk (AVaR)
* expected tail loss (ETL).


Let $ x_c = P_0 (1 + \mu + \sigma z_c) $

$$ \text{Expected Shortfall} = P_0 - \frac{1}{c} \int_{-\infty}^{x_c} xf(x) dx $$

, where $$ \tilde{X} = P_0 \times (1+\tilde{r})  \sim N( P_0 (1+\mu) , P_0 \sigma^2)$$

Substitute, 

$$ \text{Expected Shortfall} = P_0 \times (\frac{1}{c} \phi (\Phi^{-1} (c)) \sigma - \mu )  $$
, where $\phi(x)$ denotes the standard normal density function and $\Phi(\alpha)$ the $\alpha$ quantile of the standard normal distribution.  

In [0]:
#stats.norm.pdf()
def ES(P0,mu,sigma,c):
  zc = stats.norm.ppf(c,loc=0,scale=1)
  ESout = P0 * ( stats.norm.pdf(zc,loc=0,scale=1)/c * sigma - mu) 
  return ESout

print(VaR(P0,mu,sigma,0.05))
print(ES(P0,mu,sigma,0.05))
 

27690.25109372246
35007.670068351305


In [0]:
## Monte Carlo Simulation
allES = []
for i in range(1000):
  mont1 = pd.Series(np.random.normal(loc=mu,scale=sigma,size=1000))
  var = mont1.quantile(0.05)
  ES = mont1[mont1<var].mean()*P0
  allES.append(ES)
  #print(var)

np.mean(allES)
#np.mean(allES)

-34927.74811875809